In [21]:
#from pathlib import Path

path_data = "/data"
path_original_images = 'data/original_images'
path_processed_images = "/".join([path_data, "processed_imgs"])


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd

import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)


plt.style.use('ggplot')

Torch version: 1.10.2+cu113


In [14]:
data = pd.read_csv('data/images_v2.csv')
data = data[data.id != '5d4befbfcf0b3a0f3f353c2e'] #remove rows with corrupted image 
data.head()

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
0,5d4be6cecf0b3a0f3f344586,Burundi,bi,af,image,assets/5d4be6cecf0b3a0f3f344586/5d4be6cecf0b3a...,icons,butoyi,26.994581
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581


In [15]:
#  list of subjective topics
subjective_topics = ['favorite home decorations', 'favourite item in kitchen', 
                    'favourite sports clubs', 'how the most loved item is used',
                    'icons', 'idols', 'latest furniture bought', 'looking over the shoulder',
                    'most loved item', 'most loved toy', 'most played songs on the radio',
                    'music idol', 'next big thing you are planning to buy',
                     'playing with most loved toy',
                    'thing i dream about having','things i wish i had',
                     'using most loved item', 'youth culture', 'what i wish i could buy']

In [16]:
templates = [
    'an image of {}.',
]

In [17]:
# map each topic to list of corresponding image ids
dict_topic2img = {}
dict_id2country = {}
for list_topics, image_id in zip(data['topics'], data['id']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in subjective_topics: # remove subjectives
            if topic not in dict_topic2img:
                dict_topic2img[topic] = set()
            dict_topic2img[topic].add(image_id)

print(len(dict_topic2img))

270


In [18]:
list_img_path = []
list_topic = []

for topic, img_lst in dict_topic2img.items():
    for path in img_lst:
        list_img_path.append(path)
        list_topic.append(topic)
    

In [19]:
len(list_img_path)

46133

In [23]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-h59kz773
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-h59kz773
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33


In [24]:
import clip
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [25]:
model, preprocess = clip.load("ViT-B/32")


model.cuda().eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [27]:
import tqdm
from pathlib import Path

# processed_images = [path_processed_images + "/" + img_id + ".npy" for img_id in data['id']]
image_names = list(dict_topic2img.keys())
count = 10
end = len(image_names) // count + 1
similarity_CLIP = []

total_proc_imgs = 0
for k in range(0, end):
    print(k)                    
    imgs_corrupted = []
    images_prep = []
    
    texts = image_names[count*k:count*(k+1)]
    image_ids = [list(dict_topic2img[topic]) for topic in texts]
    texts_k = [[topic] * len(list(dict_topic2img[topic])) for topic in texts]
#     print(len(texts_k), len(image_ids))
    flat_image_ids = [item for sublist in image_ids for item in sublist]
    flat_texts = [item for sublist in texts_k for item in sublist]
    texts = flat_texts
    k_processed_images = [path_processed_images + "/" + img_id + ".npy" for img_id in flat_image_ids]
    print(len(texts), len(k_processed_images))
#     break
    
    for image_path in tqdm.tqdm(k_processed_images):
        if Path(image_path).is_file():         
            images_prep.append(torch.from_numpy(np.load(image_path)).to(device))
        else:
            imgs_corrupted.append(image_path)
            print(f"{image_path} corrupted")
            
    images_prep = torch.stack(images_prep)
    torch.cuda.empty_cache()
                    
    with torch.no_grad():
        image_features = model.encode_image(images_prep)
        image_features /= image_features.norm(dim=-1, keepdim=True)

#         text_tokens = clip.tokenize([desc for desc in texts]).to(device)
        text_tokens = clip.tokenize([f"a photo of a {desc}" for desc in texts]).to(device)
        text_features = model.encode_text(text_tokens).float()
        text_features /= text_features.norm(dim=-1, keepdim=True)
                    
    similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T # dot product of image and text features
    similarity_diag = torch.from_numpy(similarity.diagonal())
    
    similarity_CLIP.append(similarity_diag)
    print(f"sym diag:{len(similarity_diag)}")
# print(total_proc_imgs)

0
3413 3413


100%|██████████| 3413/3413 [03:39<00:00, 15.56it/s]


sym diag:3413
1
4043 4043


100%|██████████| 4043/4043 [03:50<00:00, 17.53it/s]


sym diag:4043
2
3226 3226


...

sym diag:27
26
14 14


100%|██████████| 14/14 [00:00<00:00, 50.09it/s]


sym diag:14
27
0 0


0it [00:00, ?it/s]


RuntimeError: stack expects a non-empty TensorList

In [28]:
similarity_CLIP2 = torch.hstack(similarity_CLIP).cpu().numpy()
similarity_CLIP2.shape

(46133,)

In [30]:
dict_img2income = {}

for income, image_id in zip(data['income'], data['id']):
    dict_img2income[image_id] = income

In [31]:
# save CLIP scores to dataframe ..
# results_df = data
# results_df["CLIP score"] = similarity_CLIP2

image_names = list(dict_topic2img.keys())
count = 10
end = len(image_names) // count + 1
similarity_CLIP = []

results_df = pd.DataFrame()

ids = []
topics = []
income = []
for k in range(0, end):
    imgs_corrupted = []
    images_prep = []
    
    texts = image_names[count*k:count*(k+1)]
    image_ids = [list(dict_topic2img[topic]) for topic in texts]
    texts_k = [[topic] * len(list(dict_topic2img[topic])) for topic in texts]
    flat_image_ids = [item for sublist in image_ids for item in sublist]
    flat_texts = [item for sublist in texts_k for item in sublist]
    for image_id in flat_image_ids:
        ids.append(image_id)
        income.append(dict_img2income[image_id])
    
    for topic in flat_texts:
        topics.append(topic)
    
results_df['id'] = ids
results_df['topics'] = topics
results_df['income'] = income
results_df['CLIP score'] = similarity_CLIP2

results_df

,id,topics,income,CLIP score
0,5d4be7f4cf0b3a0f3f34656f,family snapshots,738.000000,0.302874
1,5ec4f913f0611d7ddd74143c,family snapshots,3195.000000,0.274862
2,5d4be8bbcf0b3a0f3f3474d2,family snapshots,311.104038,0.217679
3,5d4be94fcf0b3a0f3f34862a,family snapshots,694.000000,0.215334
4,5d4be0aacf0b3a0f3f339a57,family snapshots,5621.000000,0.275776
...,...,...,...,...
46128,5d4be620cf0b3a0f3f3431a9,baking sheets,3755.000000,0.280414
46129,5d4be623cf0b3a0f3f3431fd,baking tables,3755.000000,0.238887
46130,5d4be624cf0b3a0f3f343213,electricity wires,3755.000000,0.237936
46131,5d4be82dcf0b3a0f3f346b93,meat markets,4608.946297,0.292274


In [32]:
results_df = pd.merge(results_df, data[['id','region.id', 'country.name','imageRelPath']], on='id')

In [40]:
results_df.to_csv("data/results_CLIP_sep.csv")